## 0. Создать папку source, поместить туда файлы из приложения ( имитируем папки с источником данных)

In [1]:
import os
import shutil 
import numpy as np
import pandas as pd
from datetime import datetime
import xlrd
import re
import logging

## 1. Переместить файлы из папки source в папку data (имитируем перемещение)

In [2]:
def file_check(dir_name):
    '''Create list of all files and then delet .txt file
        all_files that file which contain only excel files.
    '''
    if os.path.exists(dir_name):
        all_files = os.listdir(dir_name)
        i = 0
        while i < len(all_files):
            match = re.search(r'\b\.txt',str(all_files[i]))
            if match:
                del all_files[i]
            i += 1
        return all_files
    else:
        print('This dirrectory doesn\'t exist.')
        

In [3]:
all_files = file_check('source')
print(all_files)

['Задача 6 CAPACITIES.xlsx', 'Задача 6 REUTERS.xls']


In [4]:
#Create a log file. In the end of programm we will change name.
logging.basicConfig(filename='log_file.log', level = logging.DEBUG)

In [5]:
#Create directory
try:
    os.mkdir('data')
except:
    print('this folder already exist')

In [6]:
def copy_files(all_files, source_directory, new_directory):
    '''Copy files in new_dirrectory '''
    for i in all_files:
        try:
            shutil.copy(r'{}\{}'.format(source_directory, i), r'{}\{}'.format(new_directory, i))
        except:
            print('This file doesn\'t exist.')
            logging.error('This files doesn\'t exist')


In [7]:
copy_files(all_files, 'source', 'data')

## 2. Прочесть подряд все excel файлы в директории data в dataframe, писать имя читаемого файла в debug

In [8]:
#Add file's names in log
all_excell_files = []
for i in all_files:
    if i.endswith('.xls') or i.endswith('.xlsx'):
        logging.debug('Add file {}'.format(i))
        all_excell_files.append(i)


In [9]:
all_excell_files

['Задача 6 CAPACITIES.xlsx', 'Задача 6 REUTERS.xls']

этот блок кода я оставил для своих нужд я пытался вызов сделать это через фукцию, но пока не понял как их вытащить из пространства имен функции


In [10]:
#Тут напишу на русском. Я хотел изначально все сделать в функции, но потом отказался от этой идеи. Сейчас я понял, как это сделать
#def read_all_excel(directory):
#    directory += '\\' 
#    for i in all_excell_files: 
#        return exec('df%s = pd.read_excel(directory + str(i))' % all_excell_files.index(i))

#read_all_excel('data')

In [11]:
# In df_list we place al dataframe files(warrning not a name, object)

df_list = []
directory = 'data\\' 
for i in all_excell_files: 
    exec('a = df%s = pd.read_excel(directory + str(i))' % all_excell_files.index(i))
    df_list.append(a)

In [12]:
df1

,"The information contained in Point Carbon's analyses, newsletters and on its website has been obtained from sources believed to be reliable. However, Point Carbon disclaims all warranties as to the accuracy, completeness or adequacy of such information."
0,Product and distribution:
1,Redistribution of all or part of the Product(s...
2,"The data, information, analysis, calculations,..."


In [13]:
df1.head()

,"The information contained in Point Carbon's analyses, newsletters and on its website has been obtained from sources believed to be reliable. However, Point Carbon disclaims all warranties as to the accuracy, completeness or adequacy of such information."
0,Product and distribution:
1,Redistribution of all or part of the Product(s...
2,"The data, information, analysis, calculations,..."


## 3. Если в файле нет одной из колонок:
<ul>
<li>Commodity</li>
<li>Cause</li> 
<li>Region</li>
<li>Country / Territory</li>
<li>Company</li>
<li>Site</li>
<li>Plant No.</li>
<li>Estimated Start Date</li>
<li>Expected Annual Capacity Change (kt)</li>
<li>Total Annual Capacity (kt)</li>
<li>Swing Capable</li>
выдать сообщение статуса error с помощью библиотеки logging.<b> 
Сообщение должно быть достаточно информативным. (не просто "ахтунг, тут ошибка")</b>
<br/>
4 . Если колонки присутствуют переместить файл из директории data в директорию valid_data, записав имя перемещаемого файла в debug
<br/>
<p>5. Если колонки отсутствовали переместить файл из директории data в директорию invalid_data, записав имя перемещаемого файла в debug</p>

In [14]:
column_tuple = ('Commodity', 'Cause', 'Region', 'Country / Territory','Company', 'Site', 'Plant No.', 'Estimated Start Date',
                'Expected Annual Capacity Change (kt)', 'Total Annual Capacity (kt)', 'Swing Capable')

In [15]:
def check_column(df, cur_dir,file):
    '''Check columns in file'''
    df.head()
    for i in column_tuple:
        if i not in df.columns:
            logging.error('This is very informative error! There is no column, from column_list, \nin your file {}'.format(file))
            try:
                os.mkdir('invalid_data')
            except:
                print('this folder already exist')
            try:
                shutil.move(cur_dir + file, r'invalid_data\{}'.format(file))
                logging.debug('We add {} file in invalid_data directory'.format(file))
            except:
                logging.error('This file alredy moved or didn\'t excist')
            return file
    try:
        os.mkdir('valid_data')
    except:
        print('this folder already exist')
    try:    
        shutil.move(cur_dir + file, r'valid_data\{}'.format(file))
        logging.debug('We add {} file in valid_data directory'.format(file))
    except:
        logging.error('This file alredy moved or didn\'t excist')
    return file

In [16]:
i = 0
while i < len(df_list):
    check_column(df_list[i], 'data\\', all_excell_files[i])
    i += 1 

## 7. Записать весь лог logging в файл, поместить файл в директорию logs, имя файла должно быть log_YYYY_MM_DD_HH_MM_SS.log

In [17]:
#Я не смог изменить имя лог с помощью
#logging.basicConfig(filename='log_file.log')
now = datetime.now()
cur_time = str(now.strftime("%Y_%m-%d_%H_%M_%S"))
logging.shutdown()
os.rename('log_file.log', 'log_{}.log'.format(cur_time))